In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm


In [ ]:
encoder=torch.load('/kaggle/input/vector-pooler-128/vector_pooler_128.pth')

In [ ]:
encoder= encoder.to('cuda')

In [ ]:
encoder.eval()

In [ ]:
data=pd.read_csv('/kaggle/input/suicide-watch/Suicide_Detection.csv')

In [ ]:
data=data.iloc[:90000]
data.head()

In [ ]:
data['class']= data['class'].map({'suicide':1.0,'non-suicide':0.0})
data.head()

In [ ]:
data.value_counts('class')

In [ ]:
val_data= pd.read_csv('/kaggle/input/suicide-data-paired-for-contrastive-learning/test.csv').iloc[:15000]

In [ ]:
val_data['class']=val_data['class'].map({'suicide':1.0,'non-suicide':0.0})

In [ ]:
print(val_data.shape)
val_data.head()

In [ ]:
class inference(Dataset):
    def __init__(self,data, tokenizer,max_length=128):
        self.data=data
        self.text_column='text'
        self.tokenizer=tokenizer
        self.max_length=max_length
        
        
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        text=self.data.iloc[idx][self.text_column]
        label=self.data.iloc[idx]['class']
        inputs=self.tokenizer(text,padding='max_length',truncation=True,max_length=self.max_length,return_tensors='pt')
          
        return inputs,label

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
dataset= inference(data,tokenizer)
dataloader= DataLoader(dataset,batch_size =64,shuffle=True)
val_dataset=inference(val_data,tokenizer)
val_loader=DataLoader(val_dataset,batch_size=64,shuffle=True)

In [ ]:
class Classifier(nn.Module):
    def __init__(self, encoder):
        super(Classifier, self).__init__()
        self.encoder = encoder
        for param in self.encoder.parameters():
            param.requires_grad = False
        self.fc1 = nn.Linear(256, 128)  
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 16)
        self.fc5 = nn.Linear(16, 1)
        self.drop = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            latent = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        hidden = F.relu(self.fc1(latent))
        hidden = F.relu(self.fc2(hidden))
        hidden = self.drop(hidden)
        hidden = F.relu(self.fc3(hidden))
        hidden = F.relu(self.fc4(hidden))
        out = torch.sigmoid(self.fc5(hidden))
        return out

In [ ]:
classifier= Classifier(encoder)
classifier.to('cuda')

In [ ]:
criterion= nn.BCELoss()
opt= torch.optim.Adam(classifier.parameters(),lr=0.001)

In [ ]:
epochs=16

In [ ]:

train_losses = []
valid_losses = []

In [ ]:
for epoch in range(1, epochs + 1):
    train_loss = 0.0
    valid_loss = 0.0

    classifier.train()
    for batch in tqdm(dataloader, desc=f"Epoch {epoch}/{epochs}"):
        inputs, label = batch
        
        input_ids, attention_mask = inputs['input_ids'].squeeze(1).to('cuda'), inputs['attention_mask'].squeeze(1).to('cuda')
        label = label.float().to('cuda').type(torch.float32)  

        opt.zero_grad()
        output = classifier(input_ids, attention_mask).squeeze(1)
        loss = criterion(output, label)
        loss.backward()
        opt.step()
        train_loss += loss.item() * input_ids.size(0)  

    classifier.eval()
    for batch in val_loader:
        inputs, target = batch
        
        input_ids, attention_mask = inputs['input_ids'].squeeze(1).to('cuda'), inputs['attention_mask'].squeeze(1).to('cuda')
        target = target.float().to('cuda').type(torch.float32)  

        with torch.no_grad():
            output = classifier(input_ids, attention_mask).squeeze(1)
            loss = criterion(output, target)
            valid_loss += loss.item() * input_ids.size(0) 

    train_loss = train_loss / len(dataloader.dataset)
    valid_loss = valid_loss / len(val_loader.dataset)

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print(f'\tTraining Loss: {train_loss:.6f} \tValidation Loss: {valid_loss:.6f}')





In [ ]:
torch.save(classifier,'/kaggle/working/classifier.pth')

In [ ]:

plt.figure(figsize=(10, 5))
plt.plot(range(1, n_epochs + 1), train_losses, label='Training Loss')
plt.plot(range(1, n_epochs + 1), valid_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()